# 05_funcGLM2

这是功能像的1st GLM。

In [4]:
import os
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles

# Step 1. 

复制GLM1的文件到temp文件夹

In [5]:
folders = matchfiles(f'/home/yufengxia/Data/23BPhappy/*_BPhappy_*')
#del folders[0:12] 
Param1 = 'option_hp'
Param2 = 'gamble_hp'

for folderofsubj in folders:
    subj = folderofsubj[-4:]
    GLM_dir = Path(f'{folderofsubj}/funcGLM1/')
    GLM_dir = Path(GLM_dir)
    if GLM_dir.exists() == False:
        continue
    statsfiles = matchfiles(f'{GLM_dir}/*{Param1}_{Param2}_stats.nii*')
    !cp {statsfiles}  /home/yufengxia/Data/23BPhappy/temp
    print(statsfiles)
    


/home/yufengxia/Data/23BPhappy/20230206_BPhappy_M002/funcGLM1/M002_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230220_BPhappy_M003/funcGLM1/M003_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230227_BPhappy_M005/funcGLM1/M005_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230227_BPhappy_M006/funcGLM1/M006_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230306_BPhappy_M007/funcGLM1/M007_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230312_BPhappy_M008/funcGLM1/M008_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230312_BPhappy_M009/funcGLM1/M009_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230313_BPhappy_M010/funcGLM1/M010_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230313_BPhappy_M011/funcGLM1/M011_option_CRandEV_gamble_RPE_stats.nii.gz
/home/yufengxia/Data/23BPhappy/20230318_BPhapp

In [10]:
Param1 = 'option_hp'
Param2 = 'gamble_hp'

command = '3dttest++ -setA' 
statsfiles = matchfiles(f'/home/yufengxia/Data/23BPhappy/temp/*{Param1}_{Param2}_stats.nii*')
for files in statsfiles:
    command += ' ' + files + '\'[10]\''
print(statsfiles)
%cd /home/yufengxia/Data/23BPhappy/temp
! {command} -prefix /home/yufengxia/Data/23BPhappy/results/GLM2_{Param1}_{Param2}_gamble.dset

['/home/yufengxia/Data/23BPhappy/temp/M002_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M003_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M005_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M006_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M007_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M008_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M009_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M010_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M011_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M012_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M013_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M014_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M015_option_hp_gamble_hp_stats.nii.gz'

In [ ]:
#可视化结果
AFNI_HOME = os.getenv('AFNIDIR')
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz'
%cd /home/yufengxia/Data/23BPhappy/results/
! afni -dset GLM2_{Param1}_{Param2}.dset+tlrc.BRIK  {MNIT1}